In [ ]:
"""
Code Version resnet
Apply code to the real data

# make image size as 200 x 200 (center crop 100x100, then resize to 200x200)
# RGB channels (resnet required)

# v3
1) class and functionalize
2) train / test has already spilted into two files

# v4
1) modify train generator
  - make it accept generate images from directory
    in two way:
      a) separate train / validation / test
      b) separate train / test (for this case, validation set should gen from training set)

# v4b
1) implement resnet with dropout in the conv layers

# v5
1) Model boosting archi

"""

In [1]:
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_VISIBLE_DEVICES=5


In [2]:
from PIL import Image
import skimage.io as skio
import glob
import numpy as np
import scipy as sp
import scipy.stats 
import pandas as pd
import os
import re
import csv
from scipy.stats import percentileofscore
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, f1_score

# basic libraries
from __future__ import print_function
import random
import matplotlib.pyplot as plt

# NN libraries
import keras
from keras.models import Sequential, Model
from keras.models import load_model, save_model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D, Conv2D, BatchNormalization, concatenate
from keras.utils import np_utils
from keras import backend as K
from keras.optimizers import SGD, Adam, Adagrad
from keras.regularizers import l1, l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback
from keras.applications import resnet50
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import identity_block
from keras.applications.resnet50 import conv_block
from keras.applications.resnet50 import preprocess_input
from keras.layers import AveragePooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

K.set_image_dim_ordering('tf')

Using TensorFlow backend.


In [3]:
"""
self function calling
"""
%run -i 'bin/callbacks_miscs.py'
%run -i 'bin/py_init_data.py'
%run -i 'bin/py_generator_for_model.py'
#from keras.applications.resnet50 import _obtain_input_shape

#%run -i 'bin/resnet_with_drp.py'


In [4]:
to_get = 'activation_'
layer_info = ['none', # 0
              "max_pooling2d",  # 1
              to_get + '10',  #2
              to_get + '22',  # 3
              to_get + '40',  # 4
              'global_avg_pooling2d_1' # 5
             ] 
layer_info[1:2]
layer_info[1: 4]

['max_pooling2d', 'activation_10', 'activation_22']

In [9]:
def resnet_model_build(resnet_model, 
                       use_stage, 
                       freeze_stage, acti):
    
    for layer in resnet_model.layers:
        layer.trainable = True
    #resnet_model.summary()

    # design for using different activation function that will change the layer name
    if acti == 'relu':
        to_get = 'activation_'
    else:
        to_get = acti + "_"
        
    layer_info = ['none', # 0
                  "max_pooling2d",  # 1
                  to_get + '10',  #2
                  to_get + '22',  # 3
                  to_get + '40',  # 4
                  'global_avg_pooling2d_1' # 5
                 ]  
    get_layer = layer_info[use_stage]
    
        
    if freeze_stage == 1:
        free_layer_num = 5
    elif freeze_stage == 2:
        free_layer_num = 37
    elif freeze_stage == 3:
        free_layer_num = 79
    elif freeze_stage == 4:
        free_layer_num = 141
    else:
        free_layer_num = 176

    if freeze_stage == 0:
        print('all parameter tunable')
    else:
        for layer in resnet_model.layers[:free_layer_num]:
            layer.trainable = False
        
    if use_stage != 5:    
        x = resnet_model.get_layer(get_layer).output
        #x = AveragePooling2D((13, 13), name='avg_pool')(x)
        #x = Flatten()(x)
        x = GlobalAveragePooling2D()(x)
    else:
        x = resnet_model.get_layer(get_layer).output
      
    out = Dense(2, activation="softmax", name = "output")(x)
        
    model_final = Model(inputs = [resnet_model.input], outputs = [out])
    return model_final

In [6]:
# config -- put parameters here
from keras.preprocessing.image import ImageDataGenerator

### fold number and naming
#i_fold = 1 # fold number on naming
model_output_prefix = 'resnetMean_DateCut10_databoosting_stable' # Remember to modify the parameter below, this line is only about file naming
### controling parameters
n_gpu_use = 1 # this should compatible with CUDA_VISIBLE_DEVICES number

### model related parameters
fs = 0 # layer to freeze 
us = 4 # layers to dump
lr = 0.00017 # learning rate at begin
drp = 0 # dropout ratio in the Resnet Conv layers
batch_size = 64 * n_gpu_use
use_merge = False # Ignore it, design to merge meta-data
mini_batch_method = "shuffle" # shuffle or random
nn_activation = 'relu' # activation type in the resnet (default should be relu, option: relu / leakyrelu / elu)
dataset_mean_ratio = 1 # it matter if use dataset mean

### data information
dir_out_csv = '/home/seanyu/project/CCP/res_csv/' # result csv output location
dir_out_model = '/home/seanyu/project/CCP/model/' # model output location

### data initialize parameters
# dir_train: training set location
# dir_valid: validation set location (if leave blank, automatically get val set from training set by valid_ratio)
# dir_test: testing set location
data_params = {
            'dir_train': {'d_class0': '/data/put_data/seanyu/ccp/clean_date_cut/thres10/non_copper_train/',
                                'd_class1': '/data/put_data/seanyu/ccp/clean_date_cut/thres10/copper_train/'
                 },
            # leave white space " " as value if there is no validation dir
            'dir_valid': {'d_class0': '',
                                'd_class1': ''
                 },
            'dir_test': {'d_class0': '/data/put_data/seanyu/ccp/clean_date_cut/thres10/non_copper_test/',
                               'd_class1': '/data/put_data/seanyu/ccp/clean_date_cut/thres10/copper_test/'
                 },
            'valid_ratio' : 0.1
        }

### model input information
# tags: is copper defect or not (Y: copper, N: non-copper, watchout: the ordering trap ... alphabet ordering)
# crop_w/h: crop size from input image
# img_w/h: image size for the model (resizing)
# img_channels: RGB = 3
# use_self_improc: True (-selfmean) / False (-imagenet mean) / dataset (-dataset mean)
generator_params_dict = {'tags' : ['N', 'Y'],
                                 'crop_w': 100,
                                 'crop_h': 100,
                                 'img_w': 200,
                                 'img_h': 200,
                                 'img_channels': 3,
                                 'use_self_improc' : False # True / False / 'dataset'
                                }

### parameters for train generator
# parameters reference: https://keras.io/preprocessing/image/
datagen = ImageDataGenerator(
        rotation_range=45,
        width_shift_range=0.25,
        height_shift_range=0.25,
        shear_range=0,
        zoom_range=[0.2, 2.3],
        horizontal_flip=True, vertical_flip = True,
        fill_mode='wrap')

### parameters for validation augmentation
#datagen_val = None

datagen_val = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0,
    zoom_range=[0.5, 1.5],
    horizontal_flip=True, vertical_flip = True,
    fill_mode='wrap')


In [7]:
i_fold = 3

# Initialize the data
data_cla = init_data_from_directory(data_params)
train_nonC, val_nonC, test_nonC, train_C, val_C, test_C = data_cla.get_train_val_test_df()
print('Non_copper training/validation/testing ' + str(len(train_nonC)) + "/" + str(len(val_nonC)) + "/" + str(len(test_nonC)))
print('copper training/validation/testing ' + str(len(train_C)) + "/" + str(len(val_C)) + "/" + str(len(test_C)))

# Check table independcy here (should be empty!, if not empty, it means data contamination)
if len(set(train_C.pid).intersection(test_C.pid) ) != 0:
    print('die')
    raise 'YOU MUST ERROR HERE!'
if len(set(train_nonC.pid).intersection(test_nonC.pid) ) != 0:
    print('die')
    raise 'YOU MUST ERROR HERE!'

        
# Get training set mean of rgb
if generator_params_dict['use_self_improc'] == 'dataset':
    print('use dataset mean')
    avg_dataset = get_training_set_mean(df_class0= train_C, df_class1= train_nonC, n_core=8)
    generator_params_dict['dataset_mean'] = avg_dataset / np.float32(dataset_mean_ratio)
    # write the self_mean information to a txt file
    csv_file_name = dir_out_model + "/rgbConfig_" + opt + ".txt"
    np.savetxt(csv_file_name, avg_dataset)
else:
    generator_params_dict['dataset_mean'] = None
    print('do not use dataset mean')  

gen_data = call_generators(generator_params_dict, dta_gen= datagen)

x_val, y_val = gen_data.get_validation_data(df_class0= val_nonC, df_class1= val_C, class_0_ratio = 1,  use_im_gen = datagen_val, n_gen_loop = 3)

# shuffle dataframe
train_C = train_C.sample(frac=1).reset_index(drop = True)
train_nonC = train_nonC.sample(frac=1).reset_index(drop = True)

use inner split k-folds
Non_copper training/validation/testing 529295/58811/76704
copper training/validation/testing 7587/843/977
do not use dataset mean
1686
(6744, 200, 200, 3)
use imagenet mean


In [10]:
"""
model testing building block
"""
K.clear_session()
generation = 1
resnet_model = ResNet50(include_top=False, weights = "imagenet", input_shape = (200, 200, 3), pooling ='avg')
model = resnet_model_build(resnet_model, freeze_stage= fs, use_stage= us, acti = nn_activation)
model.summary()

all parameter tunable
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200, 200, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 206, 206, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 100, 100, 64)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 100, 100, 64)  256         conv1[0][0]                      
_____________________________________________________________________

In [12]:
### should implement hard-mining
### for each generation, reload trained model, reset all parameter, change trainable data list
nb_epoch = 150 # numbers of epoch for the training process
n_batch = 400 # numbers of updates per epoch

glist = [1, 2, 3]
for generation in glist:
    print('We are now at generation' + str(generation))
    K.clear_session()
    if generation == 1:
        resnet_model = ResNet50(include_top=False, weights = "imagenet", input_shape = (200, 200, 3), pooling ='avg')
        model = resnet_model_build(resnet_model, freeze_stage= fs, use_stage= us, acti = nn_activation)
        model.summary()
        opt = 'dBoosting_thresh10_gen' + str(generation) + '_k' + str(i_fold)
        model_file_name = "model/model_" + opt + ".h5"
    else:
        print('loading exist model: ' + model_file_name)
        model = load_model(model_file_name) # load last generation model, update model_name after loading
        opt = 'dboosting_thresh10_gen' + str(generation) + '_k' + str(i_fold)
        model_file_name = "model/model_" + opt + ".h5"

    lr = 0.00017
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6)
    myoptimizer = Adam(lr= lr)
    model.compile(loss='binary_crossentropy', optimizer=myoptimizer, metrics=['acc'])

    earlystop = EarlyStopping(monitor= 'val_loss', 
                                  min_delta= 0.0001, 
                                  patience= nb_epoch / 10, 
                                  verbose=0, mode='auto')
    checkpoint = ModelCheckpoint(model_file_name,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='auto')
    loss_history = LossHistory()
    
    history_model = model.fit_generator(gen_data.train_generator(df_class0= train_nonC, 
                                                                 df_class1=train_C,
                                                                 class_0_ratio = 1,
                                                                 class_1_ratio = 1,
                                                                 bz = batch_size),
                                            steps_per_epoch = n_batch,
                                            epochs= nb_epoch,
                                            validation_data=(x_val, y_val),
                                            callbacks = [reduce_lr,
                                                         loss_history, 
                                                         checkpoint, 
                                                         earlystop
                                                         #LogAUC(), 
                                                         #f1sc()  
                                                        ])
    # save training process
    train_loss = history_model.history.get("loss")
    train_acc = history_model.history.get("acc")

    val_loss = history_model.history.get("val_loss")
    val_acc = history_model.history.get("val_acc")
    val_auc = history_model.history.get("val_auc")
    val_f1 = history_model.history.get('val_f1sc')
    val_tp = np.array(history_model.history.get('val_tp')).astype('float32')
    val_tn = np.array(history_model.history.get('val_tn')).astype('float32')
    val_fp = np.array(history_model.history.get('val_fp')).astype('float32')
    val_fn = np.array(history_model.history.get('val_fn')).astype('float32')

    pd_tmp = pd.DataFrame({'train_loss': train_loss,
                           'valid_loss': val_loss,
                           'train_acc': train_acc,
                           'valid_acc': val_acc,
                           'valid_f1': val_f1,
                           'valid_auc': val_auc,
                           'valid_TP': val_tp,
                           'valid_TN': val_tn,
                           'valid_FP': val_fp,
                           'valid_FN': val_fn})
    pd_tmp.to_csv(opt + '_training_process_gen' + str(generation) + '.csv')

    # make prediction
    
    pred_out = gen_data.model_predict_testing(model_name = model_file_name, 
                                              df_class0 = test_nonC, 
                                              df_class1 = test_C, 
                                              testing_batch_size= 12500)
    pred_out.to_csv('res_csv/testing_' + opt + '.csv', index = False)
    
    ###
    # do hard-negative-mining #
    # random predict negative samples (non-copper)? or do all prediction
    ###
    print('doing training set prediction')
    
    if generation < glist[-1]:
        mining = gen_data.model_predict_testing(model_name = model_file_name,
                                                df_class0 = train_nonC_list[generation + 1],
                                                df_class1 = train_C,
                                                testing_batch_size = 12500)
        ### stratified sampling
        xxx = pd.concat([mining[(mining['y_true'] == 0) & (mining['y_pred'] >= 0.5) & (mining['y_pred'] < 0.95)],
                 mining[(mining['y_true'] == 0) & (mining['y_pred'] >= 0.4) & (mining['y_pred'] < 0.5)].sample(frac = 0.8),
                 mining[(mining['y_true'] == 0) & (mining['y_pred'] >= 0.3) & (mining['y_pred'] < 0.4)].sample(frac = 0.6),
                 mining[(mining['y_true'] == 0) & (mining['y_pred'] >= 0.2) & (mining['y_pred'] < 0.3)].sample(frac = 0.4),
                 mining[(mining['y_true'] == 0) & (mining['y_pred'] >= 0.0) & (mining['y_pred'] < 0.2)].sample(frac = 0.2)])
        
        train_nonC_list[generation + 1] = train_nonC_list[generation + 1][train_nonC_list[generation + 1].im_path.isin(list(xxx.png_name))]
        
        # if want to keep original data (rather than purely use new fold as training samples)
        train_nonC_list[generation + 1] = pd.concat([train_nonC_list[generation], train_nonC_list[generation + 1]]) 
        
        train_nonC_list[generation + 1].reset_index(drop = True)
        print('Next generation nonC size: ' +  str(train_nonC_list[generation + 1].shape))
    else:
        print('Generation' + str(generation) + 'is the last one: done')
    

We are now at generation1
all parameter tunable
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200, 200, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 206, 206, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 100, 100, 64)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 100, 100, 64)  256         conv1[0][0]                      
___________________________________________

Epoch 1/150
400/400 [==============================] - 387s - loss: 0.1496 - acc: 0.9437 - val_loss: 0.0767 - val_acc: 0.9730
Epoch 2/150
400/400 [==============================] - 315s - loss: 0.0960 - acc: 0.9646 - val_loss: 0.0557 - val_acc: 0.9829
Epoch 3/150
400/400 [==============================] - 314s - loss: 0.0909 - acc: 0.9664 - val_loss: 0.0762 - val_acc: 0.9712
Epoch 4/150
400/400 [==============================] - 316s - loss: 0.0793 - acc: 0.9708 - val_loss: 0.0497 - val_acc: 0.9809
Epoch 5/150
400/400 [==============================] - 340s - loss: 0.0756 - acc: 0.9732 - val_loss: 0.1857 - val_acc: 0.9411
Epoch 6/150
400/400 [==============================] - 318s - loss: 0.0730 - acc: 0.9740 - val_loss: 0.1010 - val_acc: 0.9675
Epoch 7/150
400/400 [==============================] - 323s - loss: 0.0732 - acc: 0.9725 - val_loss: 0.0805 - val_acc: 0.9718
Epoch 8/150
400/400 [==============================] - 322s - loss: 0.0672 - acc: 0.9753 - val_loss: 0.0550 - val_acc:

400/400 [==============================] - 319s - loss: 0.0367 - acc: 0.9877 - val_loss: 0.0297 - val_acc: 0.9893
Epoch 33/150
400/400 [==============================] - 318s - loss: 0.0324 - acc: 0.9884 - val_loss: 0.0265 - val_acc: 0.9913
Epoch 34/150
400/400 [==============================] - 318s - loss: 0.0348 - acc: 0.9872 - val_loss: 0.0275 - val_acc: 0.9901
Epoch 35/150
400/400 [==============================] - 317s - loss: 0.0349 - acc: 0.9878 - val_loss: 0.0314 - val_acc: 0.9901
Epoch 36/150
400/400 [==============================] - 315s - loss: 0.0342 - acc: 0.9879 - val_loss: 0.0267 - val_acc: 0.9917
Epoch 37/150
400/400 [==============================] - 316s - loss: 0.0298 - acc: 0.9895 - val_loss: 0.0312 - val_acc: 0.9926
Epoch 38/150
400/400 [==============================] - 317s - loss: 0.0342 - acc: 0.9875 - val_loss: 0.0318 - val_acc: 0.9905
Epoch 39/150
400/400 [==============================] - 316s - loss: 0.0295 - acc: 0.9891 - val_loss: 0.0272 - val_acc: 0.99

NameError: name 'train_nonC_list' is not defined

In [ ]:
model_file_name

In [ ]:
train_nonC_list[2].head()

In [13]:
train_nonC_list

NameError: name 'train_nonC_list' is not defined

In [ ]:
########################
# below is testing block

In [ ]:
K.clear_session()
fs = 1
us = 4

resnet_model = ResNet50(include_top=False, weights = "imagenet", input_shape = (200, 200, 3), pooling ='max')
model = resnet_model_build(resnet_model, freeze_stage= fs, use_stage= us)
model.summary()

lr = 0.000017
myoptimizer = Adam(lr= lr)
model.compile(loss='binary_crossentropy', optimizer=myoptimizer, metrics=['acc'])

batch_size = 36
nb_epoch = 100
n_batch = 300
use_merge = False


earlystop = EarlyStopping(monitor= 'val_loss', 
                          min_delta= 0.0001, 
                          patience= nb_epoch / 10, 
                          verbose=0, mode='auto')

weights_d1 = WeightHistory("dense1")
weights_d2 = WeightHistory("dense2")
weights_d3 = WeightHistory("output")
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8)

opt = 'freeze' + str(fs) + "_block-use_" + str(us) + '_dynamic_aug_pool_k' + str(1)

model_file_name = "model/model_" + opt + ".h5"
checkpoint = ModelCheckpoint(model_file_name,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')
loss_history = LossHistory()

if use_merge:
    pass
else:
    history_model = model.fit_generator(gen_data.train_generator(df_class0=train_nonC, 
                                                             df_class1=train_C,
                                                             class_0_ratio = 1,
                                                             class_1_ratio = 1,
                                                             bz = batch_size),
                                    steps_per_epoch = n_batch,
                                    epochs= nb_epoch,
                                    validation_data=(x_val, y_val),
                                    callbacks = [
                                                 #reduce_lr,
                                                 loss_history, 
                                                 checkpoint, 
                                                 earlystop,
                                                 LogAUC(), 
                                                 f1sc(), 
                                                 weights_d1, weights_d2, weights_d3])

###
# save training process
train_loss = history_model.history.get("loss")
train_acc = history_model.history.get("acc")

val_loss = history_model.history.get("val_loss")
val_acc = history_model.history.get("val_acc")
val_auc = history_model.history.get("val_auc")
val_f1 = history_model.history.get('val_f1sc')
val_tp = np.array(history_model.history.get('val_tp')).astype('float32')
val_tn = np.array(history_model.history.get('val_tn')).astype('float32')
val_fp = np.array(history_model.history.get('val_fp')).astype('float32')
val_fn = np.array(history_model.history.get('val_fn')).astype('float32')

pd_tmp = pd.DataFrame({'train_loss': train_loss,
                       'valid_loss': val_loss,
                       'train_acc': train_acc,
                       'valid_acc': val_acc,
                       'valid_f1': val_f1,
                       'valid_auc': val_auc,
                       'valid_TP': val_tp,
                       'valid_TN': val_tn,
                       'valid_FP': val_fp,
                       'valid_FN': val_fn})
pd_tmp.to_csv(opt + '_training_process.csv')

# make prediction
pred_out = gen_data.model_predict_testing(model_name = model_file_name, 
                                          df_class0 = test_nonC, 
                                          df_class1 = test_C, 
                                          testing_batch_size= 10000)
pred_out.to_csv('res_csv/testing_' + opt + '.csv', index = False)

# plot weights changes
print("handling weight plot")
plt.plot(range(len(weights_d1.rate)), weights_d1.rate, label='Dense1')
plt.plot(range(len(weights_d2.rate)), weights_d2.rate, label='Dense2')
plt.plot(range(len(weights_d3.rate)), weights_d3.rate, label='Output')
plt.title("Relative change of Weight per update")
plt.xlabel("update")
plt.legend(loc='best')
plt.savefig("res_image/weight_" + opt + ".png", dpi=700)
plt.close()



In [ ]:
test_done = pd.read_csv('res_csv/testing_' + opt + '.csv')
xal = test_done.y_true.values + np.random.rand(len(test_done.y_true.values)) / 2
plt.scatter(xal , test_done.y_pred.values, alpha=0.2, marker='.')
plt.yticks(np.arange(11)/10.0)
plt.plot([0,2], [0.1, 0.1], 'r-', alpha = 0.25)
plt.show()



In [ ]:
#test_done.y_true.values
idd = 0.05
fpr = sum((test_done.y_true == 0) & (test_done.y_pred > idd)) / np.float(sum(test_done.y_true == 0))

fnr = sum((test_done.y_true == 1) & (test_done.y_pred <= idd)) / np.float(sum(test_done.y_true == 1))

print('false-positive rate of ' + str(idd) + ' = ' + str(fpr) )
print('false-negative rate of ' + str(idd) + ' = ' +str(fnr) )

In [ ]:
K.clear_session()

fs = 1
us = 5

resnet_model = ResNet50(include_top=False, weights = "imagenet", input_shape = (200, 200, 3), pooling ='max')
model = resnet_model_build(resnet_model, freeze_stage= fs, use_stage= us)
model.summary()

lr = 0.00017
myoptimizer = Adam(lr= lr)
model.compile(loss='binary_crossentropy', optimizer=myoptimizer, metrics=['acc'])

batch_size = 36
nb_epoch = 100
n_batch = 500
use_merge = False


earlystop = EarlyStopping(monitor= 'val_loss', 
                          min_delta= 0.0001, 
                          patience= nb_epoch / 10, 
                          verbose=0, mode='auto')

weights_d1 = WeightHistory("dense1")
weights_d2 = WeightHistory("dense2")
weights_d3 = WeightHistory("output")

opt = 'freeze' + str(fs) + "_block-use_" + str(us)

model_file_name = "model/model_" + opt + ".h5"
checkpoint = ModelCheckpoint(model_file_name,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')
loss_history = LossHistory()

if use_merge:
    pass
else:
    history_model = model.fit_generator(gen_data.train_generator(df_class0=train_nonC, 
                                                             df_class1=train_C,
                                                             class_0_ratio = 1,
                                                             class_1_ratio = 1,
                                                             bz = batch_size),
                                    steps_per_epoch = n_batch,
                                    epochs= nb_epoch,
                                    validation_data=(x_val, y_val),
                                    callbacks = [loss_history, 
                                                 checkpoint, 
                                                 earlystop,
                                                 LogAUC(), 
                                                 f1sc(), 
                                                 weights_d1, weights_d2, weights_d3])

###
# save training process
train_loss = history_model.history.get("loss")
train_acc = history_model.history.get("acc")

val_loss = history_model.history.get("val_loss")
val_acc = history_model.history.get("val_acc")
val_auc = history_model.history.get("val_auc")
val_f1 = history_model.history.get('val_f1sc')
val_tp = np.array(history_model.history.get('val_tp')).astype('float32')
val_tn = np.array(history_model.history.get('val_tn')).astype('float32')
val_fp = np.array(history_model.history.get('val_fp')).astype('float32')
val_fn = np.array(history_model.history.get('val_fn')).astype('float32')

pd_tmp = pd.DataFrame({'train_loss': train_loss,
                       'valid_loss': val_loss,
                       'train_acc': train_acc,
                       'valid_acc': val_acc,
                       'valid_f1': val_f1,
                       'valid_auc': val_auc,
                       'valid_TP': val_tp,
                       'valid_TN': val_tn,
                       'valid_FP': val_fp,
                       'valid_FN': val_fn})
pd_tmp.to_csv(opt + '_training_process.csv')

# make prediction
pred_out = gen_data.model_predict_testing(model_name = model_file_name, 
                                          df_class0 = test_nonC, 
                                          df_class1 = test_C, 
                                          testing_batch_size= 10000)
pred_out.to_csv('res_csv/testing_' + opt + '.csv', index = False)

# plot weights changes
print("handling weight plot")
plt.plot(range(len(weights_d1.rate)), weights_d1.rate, label='Dense1')
plt.plot(range(len(weights_d2.rate)), weights_d2.rate, label='Dense2')
plt.plot(range(len(weights_d3.rate)), weights_d3.rate, label='Output')
plt.title("Relative change of Weight per update")
plt.xlabel("update")
plt.legend(loc='best')
plt.savefig("res_image/weight_" + opt + ".png", dpi=700)
plt.close()



In [ ]:
K.clear_session()

fs = 3
us = 5

resnet_model = ResNet50(include_top=False, weights = "imagenet", input_shape = (200, 200, 3), pooling ='max')
model = resnet_model_build(resnet_model, freeze_stage= fs, use_stage= us)
model.summary()

lr = 0.00017
myoptimizer = Adam(lr= lr)
model.compile(loss='binary_crossentropy', optimizer=myoptimizer, metrics=['acc'])

batch_size = 36
nb_epoch = 100
n_batch = 500
use_merge = False


earlystop = EarlyStopping(monitor= 'val_loss', 
                          min_delta= 0.0001, 
                          patience= nb_epoch / 10, 
                          verbose=0, mode='auto')

weights_d1 = WeightHistory("dense1")
weights_d2 = WeightHistory("dense2")
weights_d3 = WeightHistory("output")

opt = 'freeze' + str(fs) + "_block-use_" + str(us)

model_file_name = "model/model_" + opt + ".h5"
checkpoint = ModelCheckpoint(model_file_name,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')
loss_history = LossHistory()

if use_merge:
    pass
else:
    history_model = model.fit_generator(gen_data.train_generator(df_class0=train_nonC, 
                                                             df_class1=train_C,
                                                             class_0_ratio = 1,
                                                             class_1_ratio = 1,
                                                             bz = batch_size),
                                    steps_per_epoch = n_batch,
                                    epochs= nb_epoch,
                                    validation_data=(x_val, y_val),
                                    callbacks = [loss_history, 
                                                 checkpoint, 
                                                 earlystop,
                                                 LogAUC(), 
                                                 f1sc(), 
                                                 weights_d1, weights_d2, weights_d3])

###
# save training process
train_loss = history_model.history.get("loss")
train_acc = history_model.history.get("acc")

val_loss = history_model.history.get("val_loss")
val_acc = history_model.history.get("val_acc")
val_auc = history_model.history.get("val_auc")
val_f1 = history_model.history.get('val_f1sc')
val_tp = np.array(history_model.history.get('val_tp')).astype('float32')
val_tn = np.array(history_model.history.get('val_tn')).astype('float32')
val_fp = np.array(history_model.history.get('val_fp')).astype('float32')
val_fn = np.array(history_model.history.get('val_fn')).astype('float32')

pd_tmp = pd.DataFrame({'train_loss': train_loss,
                       'valid_loss': val_loss,
                       'train_acc': train_acc,
                       'valid_acc': val_acc,
                       'valid_f1': val_f1,
                       'valid_auc': val_auc,
                       'valid_TP': val_tp,
                       'valid_TN': val_tn,
                       'valid_FP': val_fp,
                       'valid_FN': val_fn})
pd_tmp.to_csv(opt + '_training_process.csv')

# make prediction
pred_out = gen_data.model_predict_testing(model_name = model_file_name, 
                                          df_class0 = test_nonC, 
                                          df_class1 = test_C, 
                                          testing_batch_size= 10000)
pred_out.to_csv('res_csv/testing_' + opt + '.csv', index = False)

# plot weights changes
print("handling weight plot")
plt.plot(range(len(weights_d1.rate)), weights_d1.rate, label='Dense1')
plt.plot(range(len(weights_d2.rate)), weights_d2.rate, label='Dense2')
plt.plot(range(len(weights_d3.rate)), weights_d3.rate, label='Output')
plt.title("Relative change of Weight per update")
plt.xlabel("update")
plt.legend(loc='best')
plt.savefig("res_image/weight_" + opt + ".png", dpi=700)
plt.close()



In [ ]:
import sys
orig_stdout = sys.stdout

ff = open("model_summary.txt", "w")
sys.stdout = ff
print(model.summary())
print("------------------")
sys.stdout = orig_stdout
ff.close()

In [ ]:
import seaborn as sns
ax = sns.heatmap(pred_test, linewidths= 0.5, cmap = 'YlGnBu',
                yticklabels = pd_test.tag.values)
plt.savefig('test_outcome.png')
plt.show()

In [ ]:
"""
Cross validation for the froze1, use4
"""

In [ ]:
np.arange(1,11)

In [ ]:
for i_c in np.arange(1, 11):
    print('round' + str(i_c))
    print('init_paramters')
    data_params = {
           'non_copper_meta_all': '/home/gtx980/Desktop/CCP/Data/DataSet/Data_CSV/Non_Copper_all(304455).csv',
           'copper_meta_all': '/home/gtx980/Desktop/CCP/Data/DataSet/Data_CSV/Copper_all(4582).csv',
           'non_copper_testing': '/home/gtx980/Desktop/CCP/Data/DataSet/Data_CSV/non_copper_test_k' + str(i_c) + '.csv',
           'copper_testing': '/home/gtx980/Desktop/CCP/Data/DataSet/Data_CSV/copper_test_k' + str(i_c) + '.csv',
           'non_copper_img_folder' : '/home/gtx980/Desktop/CCP/Data/DataSet/All/',
           'copper_img_folder': '/home/gtx980/Desktop/CCP/Data/DataSet/Copper/',
           'valid_ratio' : 0.1}

    generator_params_dict = {'tags' : ['N', 'Y'],
                             'crop_w': 100,
                             'crop_h': 100,
                             'img_w': 200,
                             'img_h': 200,
                             'img_channels': 3
                            }

    datagen = ImageDataGenerator(
            rotation_range=90,
            width_shift_range=0.15,
            height_shift_range=0.15,
            shear_range=0.,
            zoom_range=0.,
            horizontal_flip= True,
            vertical_flip = True,
            fill_mode='nearest')

    data_cla = init_data(data_params)
    train_nonC, val_nonC, test_nonC, train_C, val_C, test_C = data_cla.get_train_val_test_df()

    gen_data = call_generators(generator_params_dict, dta_gen= datagen)
    
    x_val, y_val = gen_data.get_validation_data(df_class0= val_nonC, 
                                                df_class1= val_C)
    print(data_params['copper_testing'])
    print(data_params['non_copper_testing'])
    print('Data reading done, clear session')
    K.clear_session()
    fs = 1
    us = 4

    resnet_model = ResNet50(include_top=False, weights = "imagenet", input_shape = (200, 200, 3), pooling ='max')
    model = resnet_model_build(resnet_model, freeze_stage= fs, use_stage= us)
    model.summary()

    lr = 0.00017
    myoptimizer = Adam(lr= lr)
    model.compile(loss='binary_crossentropy', optimizer=myoptimizer, metrics=['acc'])

    batch_size = 36
    nb_epoch = 150
    n_batch = 500
    use_merge = False


    earlystop = EarlyStopping(monitor= 'val_loss', 
                              min_delta= 0.0001, 
                              patience= nb_epoch / 10, 
                              verbose=0, mode='auto')

    weights_d1 = WeightHistory("dense1")
    weights_d2 = WeightHistory("dense2")
    weights_d3 = WeightHistory("output")

    opt = 'freeze' + str(fs) + "_block-use_" + str(us) + 'k_' + str(i_c)

    model_file_name = "model/model_" + opt + ".h5"
    checkpoint = ModelCheckpoint(model_file_name,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='auto')
    loss_history = LossHistory()

    if use_merge:
        pass
    else:
        history_model = model.fit_generator(gen_data.train_generator(df_class0=train_nonC, 
                                                                 df_class1=train_C,
                                                                 class_0_ratio = 1,
                                                                 class_1_ratio = 1,
                                                                 bz = batch_size),
                                        steps_per_epoch = n_batch,
                                        epochs= nb_epoch,
                                        validation_data=(x_val, y_val),
                                        callbacks = [loss_history, 
                                                     checkpoint, 
                                                     earlystop,
                                                     LogAUC(), 
                                                     f1sc(), 
                                                     weights_d1, weights_d2, weights_d3])

    ###
    # save training process
    train_loss = history_model.history.get("loss")
    train_acc = history_model.history.get("acc")

    val_loss = history_model.history.get("val_loss")
    val_acc = history_model.history.get("val_acc")
    val_auc = history_model.history.get("val_auc")
    val_f1 = history_model.history.get('val_f1sc')
    val_tp = np.array(history_model.history.get('val_tp')).astype('float32')
    val_tn = np.array(history_model.history.get('val_tn')).astype('float32')
    val_fp = np.array(history_model.history.get('val_fp')).astype('float32')
    val_fn = np.array(history_model.history.get('val_fn')).astype('float32')

    pd_tmp = pd.DataFrame({'train_loss': train_loss,
                           'valid_loss': val_loss,
                           'train_acc': train_acc,
                           'valid_acc': val_acc,
                           'valid_f1': val_f1,
                           'valid_auc': val_auc,
                           'valid_TP': val_tp,
                           'valid_TN': val_tn,
                           'valid_FP': val_fp,
                           'valid_FN': val_fn})
    pd_tmp.to_csv(opt + '_training_process.csv')

    # make prediction
    pred_out = gen_data.model_predict_testing(model_name = model_file_name, 
                                              df_class0 = test_nonC, 
                                              df_class1 = test_C, 
                                              testing_batch_size= 10000)
    pred_out.to_csv('res_csv/testing_' + opt + '.csv', index = False)

    # plot weights changes
    print("handling weight plot")
    plt.plot(range(len(weights_d1.rate)), weights_d1.rate, label='Dense1')
    plt.plot(range(len(weights_d2.rate)), weights_d2.rate, label='Dense2')gen_data.model_predict_testing(model_name = model_file_name, 
                                              df_class0 = test_nonC, 
                                              df_class1 = test_C, 
                                              testing_batch_size= 10000)
    plt.plot(range(len(weights_d3.rate)), weights_d3.rate, label='Output')
    plt.title("Relative change of Weight per update")
    plt.xlabel("update")
    plt.legend(loc='best')
    plt.savefig("res_image/weight_" + opt + ".png", dpi=700)
    plt.close()



In [ ]:
plt.plot(pred_out.y_true, pred_out.y_pred, 'b.')
plt.show()

In [ ]:
"""
Quick testing area
"""

In [ ]:
#f_im_test = glob.glob("Data/DataSet_2/special/k5/*.png")
#orig = pd.read_csv("Data/DataSet/Data_CSV_2/copper_test_k5.csv")
#f_im_orig = glob.glob("Data/DataSet_2/special/k5_original/*.png")

path_cop = "Data/DataSet_2/new/Data2017/RealCopper/"
path_nor = "Data/DataSet_2/new/Data2017/NonCopper/"

new_data_cop = glob.glob(path_cop + "*.png")
new_data_nor = glob.glob(path_nor + "*.png")



In [ ]:
i_normal = list(np.random.randint(len(new_data_nor), size = 20000))
nor_get = [new_data_nor[i] for i in i_normal]

Y_all = list(np.ones(len(new_data_cop))) + list(np.zeros(20000))

X_all = new_data_cop + nor_get # copper + non_copper
print(len(X_all))

In [ ]:
#Y_all = np.ones(len(new_data_cop)) + np.zeros(20000)
len(Y_all)

In [ ]:
X_ = np.array([scipy.misc.imresize(i, size=(self.img_w, 
                                              self.img_h)) for i in X_]) # add this line to resize images
X_= X_all.reshape((len(X_train), 
                           self.img_w, self.img_h, self.channels))

In [ ]:
orig = list(orig.pid)
orig = ['Data/DataSet_2/copper_crop/' + i for i in orig]

X_t = []
Y_t = []

X_t = np.array([ np.array(Image.open(i),dtype='float32') for i in f_im_test] )
X_t = preprocess_input(X_t)

X_o = np.array([ np.array(Image.open(i).resize((200,200)),dtype='float32') for i in orig] )
X_o = preprocess_input(X_o)

X_op = np.array([ np.array(Image.open(i),dtype='float32') for i in f_im_orig] )
X_op = preprocess_input(X_op)

tmp_model = load_model("model/model_freeze1_stage-use_4_dynamic_aug_pool6_bz32_drp0.2_k5.h5")

y_pred = tmp_model.predict(X_t)
y_pred_o = tmp_model.predict(X_o)
y_pred_op = tmp_model.predict(X_op)